### MDCATH DATASET IN MACHINE LEARNING FRAMEWORK

This tutorial provides a practical example of training ML models using the mdCATH dataset in TorchMD-Net. Before you begin, please ensure that TorchMD-Net is correctly installed. You can find installation instructions and further details [here](https://torchmd-net.readthedocs.io/en/latest/installation.html). Note that the MDCATH dataloader is available starting from TorchMD-Net version 2.4.0 and later. The next cell will write and environment file, then it's possible to create the env using the following command:

```bash
mamba env create -f environment.yml
```

In [ ]:
%%writefile environment.yaml
name: torchmdnet_env
channels:
  - conda-forge
  - defaults
dependencies:
  - torchmdnet>=2.4.0

In [2]:
import os
import torch
import lightning.pytorch as pl
from torchmdnet.data import DataModule
from torchmdnet.module import LNNP
from torchmdnet.scripts.train import get_args
from lightning.pytorch.callbacks import TQDMProgressBar, ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

In [3]:
# Define the arguments
args = get_args() # default arguments by tmdnet
args = vars(args) # convert to dictionary

pargs = {
    # DATA
    'dataset': 'MDCATH',
    'dataset_arg':{
      'numAtoms': None,
      'numResidues': None,
      'pdb_list': ['1balA00', '1ce3A00', '1e8rA00'],
      'temperatures': ['348'],
      'skip_frames': 2,
      'solid_ss': None,
      },
    'dataset_root': 'data',
    # MODEL
    'model': 'tensornet',
    'embedding_dimension': 32,
    'num_layers': 0,
    'num_rbf': 8,
    'rbf_type': 'expnorm',
    'activation': 'silu',
    'cutoff_lower': 0.0,
    'cutoff_upper': 5.0,
    'max_z': 20,
    'num_epochs': 10,
    'max_num_neighbors': 48,
    'derivative': True, 
    # TRAIN
    'batch_size': 3,
    'train_size': 200, 
    'val_size': 50,
    'test_size': 100,
    'lr': 1e-3,
    'lr_metric': 'val',
    'log_dir': 'logs/',
    'check_errors': True,
    'static_shapes': False,
    'num_workers': 2,
}

# Update the default arguments with the new ones
args.update(pargs)
os.makedirs(args['log_dir'], exist_ok=True)

In [4]:
# Here MDCATH torch_geometric dataset class is used 
# If the h5 files are not present in the 'dataset_root' then they will be downloaded from HF
# The downlaoad process can take some time

data = DataModule(args)
data.prepare_data()
data.setup("fit")

Processing mdcath source: 100%|██████████| 3/3 [00:00<00:00, 135.89it/s]

train 200, val 50, test 100



/shared/antoniom/mambaforge/envs/gemini2/lib/python3.10/site-packages/torchmdnet/utils.py:221: UserWarning: 2970 samples were excluded from the dataset
  rank_zero_warn(f"{dset_len - total} samples were excluded from the dataset")


In [5]:
# Lightning wrapper for the Neural Network Potentials in TorchMD-Net
lnnp = LNNP(args, 
    prior_model=None, 
    mean=data.mean, 
    std=data.std)

In [6]:
# Callbacks, used to save model ckpts
val_loss_name = 'val_total_mse_loss'
checkpoint_callback = ModelCheckpoint(dirpath=args['log_dir'], 
                                      monitor=val_loss_name, 
                                      every_n_epochs=2, 
                                      filename=f"epoch={{epoch}}-val_loss={{{val_loss_name}:.4f}}",
                                      save_top_k=3)

In [7]:
# Logger for the training process, it will save the training logs in a csv file
csv_logger = CSVLogger(args['log_dir'], name="", version="")

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
print(f'cuda available: {torch.cuda.is_available()}')
print(f'cuda device count: {torch.cuda.device_count()}')
print(f'CUDA_VISIBLE_DEVICES ID: {os.environ["CUDA_VISIBLE_DEVICES"]}')

cuda available: True
cuda device count: 4
CUDA_VISIBLE_DEVICES ID: 0


In [9]:
# Train
trainer = pl.Trainer(strategy="auto",
                     devices=1,
                     max_epochs=args['num_epochs'], 
                     precision=args['precision'],
                     default_root_dir = args['log_dir'],
                     logger=csv_logger,
                     callbacks=[checkpoint_callback, TQDMProgressBar(refresh_rate=1)])

trainer.fit(lnnp, data, ckpt_path=None)

/shared/antoniom/mambaforge/envs/gemini2/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /shared/antoniom/mambaforge/envs/gemini2/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/shared/antoniom/mambaforge/envs/gemini2/lib/python3.10/site-packages/lightning/fabric/loggers/csv_logs.py:196: Experiment log

train 200, val 50, test 100


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

/shared/antoniom/mambaforge/envs/gemini2/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: torchmdnet_extensions::get_neighbor_pairs_bkwd: an autograd kernel was not registered to the Autograd key(s) but we are trying to backprop through it. This may lead to silently incorrect behavior. This behavior is deprecated and will be removed in a future version of PyTorch. If your operator is differentiable, please ensure you have registered an autograd kernel to the correct Autograd key (e.g. DispatchKey::Autograd, DispatchKey::CompositeImplicitAutograd). If your operator is not differentiable, or to squash this warning and use the previous behavior, please register torch::CppFunction::makeFallthrough() to DispatchKey::Autograd. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1718580525958/work/torch/csrc/autograd/autograd_not_implemented_fallback.cpp:63.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run th

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [14]:
# Test
model = LNNP.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
trainer = pl.Trainer(inference_mode=False)
trainer.test(model, data)

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/shared/antoniom/mambaforge/envs/gemini2/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /shared/antoniom/mambaforge/envs/gemini2/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/shared/antoniom/mambaforge/envs/gemini2/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dep

train 200, val 50, test 100


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test_neg_dy_l1_loss    │     4.440413475036621     │
│    test_total_l1_loss     │     4.440413475036621     │
│      test_y_l1_loss       │            0.0            │
└───────────────────────────┴───────────────────────────┘

[{'test_total_l1_loss': 4.440413475036621,
  'test_y_l1_loss': 0.0,
  'test_neg_dy_l1_loss': 4.440413475036621}]